In [224]:
import pandas as pd
import json 
import re
import numpy as np

In [225]:
df_metrics = pd.read_csv('data/2d-03_10_2018.csv')
df_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516 entries, 0 to 515
Columns: 270 entries, Current_group_location to year
dtypes: float64(220), int64(5), object(45)
memory usage: 1.1+ MB


In [226]:
df_logs =  pd.read_csv('checkpoints/logs-ranks.csv')
df_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1294 entries, 0 to 1293
Data columns (total 16 columns):
Unnamed: 0     1294 non-null int64
teamname       1294 non-null object
loss           1294 non-null float64
S              1294 non-null int64
R              1294 non-null int64
games          1294 non-null int64
tie            1294 non-null float64
win            1294 non-null float64
competition    1294 non-null object
year           1294 non-null int64
win_diff       1294 non-null float64
win_rate       1294 non-null float64
goals_diff     1294 non-null float64
loss_rate      1294 non-null float64
team           1294 non-null object
rank           53 non-null float64
dtypes: float64(8), int64(5), object(3)
memory usage: 161.8+ KB


In [227]:
df_logs.head()

,Unnamed: 0,teamname,loss,S,R,games,tie,win,competition,year,win_diff,win_rate,goals_diff,loss_rate,team,rank
0,0,-humboldt,4.0,0,13,4,0.0,0.0,WorldCup,2005,-1.0,0.0,-3.250000,1.0,humboldt,NaN
1,1,chagamma,5.0,1,25,5,0.0,0.0,WorldCup,2003,-1.0,0.0,-4.800000,1.0,chagamma,NaN
2,2,og,4.0,1,11,4,0.0,0.0,JapanAutumnCamp,2010,-1.0,0.0,-2.500000,1.0,og,NaN
3,3,_utaustinvilla,1.0,0,6,1,0.0,0.0,USOpens,2004,-1.0,0.0,-6.000000,1.0,utaustinvilla,NaN
4,4,ahua,3.0,1,11,3,0.0,0.0,ChinaOpens,2007,-1.0,0.0,-3.333333,1.0,ahua,NaN


In [228]:
df_logs['team'] = [x.strip('_-') for x in df_logs['team']]

In [229]:
df_logs['competition'] = df_logs['competition'].str.replace(u'JapanAutumnCamp','AutumnCamp')

In [230]:
df_logs.to_csv('data/logs_extracted.csv')

In [231]:
df_logs['team'] = df_logs['team'].str.lower()

In [232]:
df_logs = df_logs.groupby(['year','competition','team']).sum().reset_index()

In [233]:
df_logs.shape[0]

1291

In [234]:
df_logs = df_logs[~(df_logs['team'].str.len()<3)]

In [235]:
rename_dict = {a:"results_%s"%a for a in df_logs.columns if a not in ['competition','year','group_name']}


df_logs =df_logs.rename(rename_dict,axis=1)
df_logs.head()

,year,competition,results_team,results_Unnamed: 0,results_loss,results_S,results_R,results_games,results_tie,results_win,results_win_diff,results_win_rate,results_goals_diff,results_loss_rate,results_rank
0,2001,WorldCup,arvand,928,1.0,3,1,4,1.0,2.0,0.250000,0.500000,0.500000,0.250000,0.0
1,2001,WorldCup,ateam,930,1.0,9,1,4,1.0,2.0,0.250000,0.500000,2.000000,0.250000,0.0
2,2001,WorldCup,attunited01,35,1.0,0,2,1,0.0,0.0,-1.000000,0.000000,-2.000000,1.000000,0.0
3,2001,WorldCup,brainstormers01,1114,1.0,16,3,7,1.0,5.0,0.571429,0.714286,1.857143,0.142857,2.0
4,2001,WorldCup,chameleons,476,4.0,10,16,7,1.0,2.0,-0.285714,0.285714,-0.857143,0.571429,0.0


#### Load manually linked logs to program

In [236]:
df_mapped = pd.read_csv('data/group_names_mapped.csv',usecols=['competition','year','team_manual','group_name'])
df_mapped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 4 columns):
competition    528 non-null object
group_name     528 non-null object
year           528 non-null int64
team_manual    60 non-null object
dtypes: int64(1), object(3)
memory usage: 16.6+ KB


In [237]:
df_merged_mapping = df_logs.merge(df_mapped,left_on=['competition','year','results_team'],right_on=['competition','year','team_manual'],how='left')
df_merged_mapping['team_alt'] = df_merged_mapping.apply(lambda x: x.group_name if pd.notnull(x.group_name) else x.results_team ,axis=1)

In [238]:
df_merged_mapping.shape[0]

1257

In [239]:
df_merged_mapping = df_merged_mapping.drop('group_name',axis=1)

#### number manually linked programs : 

In [240]:
df_merged_mapping.shape[0]

1257

#### Number of not liked groups

### Automatically linking groups to logs

In [241]:
def is_in_group(x,):
    r= re.match(r'([a-zA-Z\-]+)',x.team_alt)
    r2 = re.match(r'([a-zA-Z\-0-9]+)',x.team_alt)
    if not r:
        return False
    return r.groups()[0].lower() in x.group_name.lower(), len(r2.groups()[0])

In [242]:
def len_in_group_dup(x):
    if x.duplicate_team < 2: 
        return True
    r= re.match(r'([a-zA-Z\-0-9]+)',x.team_alt)
    if not r:
        return False
    if r.groups()[0].lower() in x.group_name.lower():
        return len(r.groups()[0])

In [243]:
df_mix = df_metrics.merge(df_merged_mapping,on=['competition','year'])


In [244]:
df_mix = df_mix.join(df_mix.apply(is_in_group,axis=1,result_type='expand').
                     rename({0:'exist',1:'length'},axis=1))


In [245]:
df = df_mix[df_mix['exist']==True].copy()

In [246]:
df.loc[:,'max'] = df.groupby(['competition','year','group_name'])['length'].transform(max)

In [247]:
df = df[df['length']==df['max']]

In [248]:
df.drop(['length','max'],axis=1,inplace=True)

In [249]:
#df.columns = ["results_%s"%a for a in df.columns if a not in ['competition','year','group_name']] +['competition','year','group_name']

In [250]:
df1 = df_metrics.merge(df,on=['competition','year','group_name'],how='left',indicator=True)
df1[df1['_merge']=='both'].shape[0]

410

In [251]:
df1[['competition','group_name','year','team_manual','_merge']].sort_values(['year','group_name']) \
    .to_csv('checkpoints/not_found_repo_names.csv')

In [252]:
df2 = df_logs.merge(df,how='left',on=['competition','year','results_team'])
df2[(df2['group_name'].isnull())][['competition','year','results_team']].sort_values(['competition','year','results_team']).to_csv('checkpoints/not_found_team.csv')


In [253]:
df.to_csv('data/merged-logs_repo.csv')